In [3]:
import time
import numpy as np
from dataclasses import dataclass
from collections import defaultdict

import bokeh
from bokeh.io import push_notebook, show, output_notebook
from bokeh import layouts
from bokeh.plotting import figure
from bokeh.palettes import Category10

output_notebook()

Loading BokehJS ...

In [4]:
color_registry = {}
color_iter = iter(Category10[10])
def get_color(name):
    if name not in color_registry:
        color = next(color_iter)
        color_registry[name] = color
    return color_registry[name]

In [5]:
def moving_average(x, w):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[w:] - cumsum[:-w]) / float(w)

In [6]:
class DataStreamSaver(object):
    def __init__(self):
        self.data = defaultdict(lambda: defaultdict(list))
        
    def update(self, data):
        for fig_name in data:
            for glyph_name in data[fig_name]:
                self.data[fig_name][glyph_name].append(data[fig_name][glyph_name])
                
    def reset(self):
        for fig_name in self.data:
            for glyph_name in self.data[fig_name]:
                self.data[fig_name][glyph_name] = []

In [7]:
def to_data_source(y, average_width=5, max_ticks=100):
    y = np.array(y)
    y = moving_average(y, average_width)
    if y.size >= max_ticks:
        tick = int(np.ceil(y.size / max_ticks))
        y = y[::tick]
    x = np.arange(y.size)
    return bokeh.models.ColumnDataSource({
        'x': x,
        'y': y
    })

In [66]:
class LineFigure(object):
    def __init__(self, title, **kwargs):
        self.fig = figure(title=title, width=800, height=300, **kwargs)
        self.glyphs = {}
        
    def update_glyph(self, glyph_name, data_source):
#         if glyph_name in self.glyphs:
#             self.fig.renderers.remove(self.glyphs[glyph_name])

        if glyph_name not in self.glyphs:
            self.glyphs[glyph_name] = self.fig.line(
                legend_label=glyph_name,
                color=get_color(glyph_name),
                alpha=0.7,
                line_width=2,
                name=glyph_name
            )
            self.fig.legend.location = 'top_left'
#         print('update!')
        self.glyphs[glyph_name].data_source = data_source
        self.glyphs[glyph_name].trigger

In [67]:
class VisDashboard(object):
    def __init__(self, figs):
        self.figs = {fig.fig.title.text: fig for fig in figs}
        self.saver = DataStreamSaver()
        self.handle = None
        
    def _sync_figures(self):
        for fig_name in self.saver.data:
            for glyph_name in self.saver.data[fig_name]:
                data_source = to_data_source(self.saver.data[fig_name][glyph_name])
                self.figs[fig_name].update_glyph(glyph_name, data_source)
            
    def push(self):
        self._sync_figures()
        push_notebook(handle=self.handle)
        
    def show(self):
        figs = [fig.fig for fig in self.figs.values()]
        self.handle = show(layouts.column(figs), notebook_handle=True)

In [68]:
if '__file__' in globals():
    class StopExecution(Exception):
        def _render_traceback_(self):
            pass
    raise StopExecution

In [106]:
vis = VisDashboard([
    LineFigure('fig_a'),
    LineFigure('fig_b'),
])
vis.show()

In [113]:
f = vis.figs['fig_b'].fig

In [75]:
for i in range(10):
    l.data_source = to_data_source(np.sin(np.arange(20 + i)))
    l.visible = False
    
    push_notebook(handle=vis.handle)

In [65]:
f.select_one

Figure(id='2908', ...)

In [114]:
l = f.line(name='myline')

In [120]:
l.data_source = to_data_source(np.random.randn(30))

In [133]:
l = f.select_one({'name': 'myline'})
l.data_source = to_data_source(np.random.randn(30))

In [134]:
push_notebook(handle=vis.handle)

In [13]:
for j in range(10):
    for i in range(60):
        vis.saver.update({'fig_a': {'gly_a': j ** 2 + i}})
        vis.saver.update({'fig_b': {'gly_a': np.log(i * j + 1), 'gyl_b': np.log(i * j * 2 + 1)}})

    vis.push()

In [13]:
# y = np.array(vis.saver.data['fig_a']['gly_a'])

# y = y[::int(y.size / vis.saver.max_ticks)]
# moving_average(y, 5)

# vis.saver.to_data_source('fig_a', 'gly_a').data